# Tutorial 1: Introduction to PyPulseq

Welcome to the PyPulseq tutorial series! This first notebook introduces the core concepts
and building blocks you need to start creating MRI pulse sequences with PyPulseq.

## What is Pulseq?

[Pulseq](https://pulseq.github.io/) is an open-source, vendor-independent file format for
MRI pulse sequence programming. Instead of writing sequences in vendor-specific languages
(e.g., Siemens IDEA, GE EPIC, Bruker ParaVision), you describe your sequence in a `.seq`
file that can be executed on any scanner with a Pulseq interpreter.

## What is PyPulseq?

**PyPulseq** is a Python package for creating Pulseq `.seq` files. It provides functions to
define RF pulses, gradient waveforms, ADC readouts, and delays — and to assemble them into
complete pulse sequences.

## What you will learn

In this notebook, you will learn:

1. How to install and import PyPulseq
2. How to define scanner hardware limits with `pp.Opts`
3. How to create a `Sequence` object
4. How to create basic sequence events (RF pulse, ADC, delay)
5. How to assemble events into blocks and inspect timing
6. How to check, plot, and export the sequence
7. How to investigate the `.seq` file format

---
## 1. Installation

There are several ways to install PyPulseq. The recommended and most convenient way is to install the latest stable release version from [PyPI](https://pypi.org/project/pypulseq/) using `pip`:

```bash
pip install pypulseq
```

If you prefer to use conda, you can install the latest stable release version from [conda-forge](https://anaconda.org/conda-forge/pypulseq) using `conda`:
```bash
conda install -c conda-forge pypulseq
```

If you want to install the latest development version, you can install it directly from the GitHub repository:
```bash
pip install git+https://github.com/pypulseq/pypulseq.git@<branch_name>
```

In [ ]:
import importlib

if not importlib.util.find_spec('pypulseq'):
    %pip install pypulseq

---
## 2. Importing PyPulseq

By convention, we import PyPulseq under the alias `pp`. We usually also import NumPy as `np`, which is used throughout for numerical operations.

In [ ]:
import numpy as np

import pypulseq as pp

print(f'PyPulseq version: {pp.__version__}')

---
## 3. Scanner Hardware Limits: `pp.Opts`

Every MRI scanner has physical limits on how fast gradients can change (slew rate), how
strong they can be (maximum gradient amplitude), how long the gaps between individual events of different types should be
(dead times), and on which time grid events should be scheduled (raster times). Before creating any sequence events, we
define these limits in an [Opts](../../src/pypulseq/opts.py) object.

This ensures that all events we create as well as the concatenation of events (the sequence) will respect the scanner's capabilities.

Typical values we set for individual scanners are:

| Parameter | Description | Typical value |
|---|---|---|
| `max_grad` | Maximum gradient amplitude | 24–80 mT/m |
| `max_slew` | Maximum gradient slew rate | 100–200 T/m/s |
| `rf_dead_time` | Dead time before RF pulse | 100 µs |
| `rf_ringdown_time` | Ringdown time after RF pulse | 10–30 µs |
| `adc_dead_time` | Dead time before ADC readout | 10 µs |

You can find a complete documentation of all possible arguments in the docstring of the [Opts](../../src/pypulseq/opts.py) class.

In [ ]:
system = pp.Opts(
    max_grad=28,
    grad_unit='mT/m',
    max_slew=150,
    slew_unit='T/m/s',
    rf_ringdown_time=20e-6,
    rf_dead_time=100e-6,
    adc_dead_time=10e-6,
)

print(f'Max gradient: {system.max_grad / system.gamma * 1e3:.1f} mT/m')
print(f'Max slew rate: {system.max_slew / system.gamma:.1f} T/m/s')
print(f'Gradient raster time: {system.grad_raster_time * 1e6:.1f} µs')
print(f'RF raster time: {system.rf_raster_time * 1e6:.1f} µs')

---
## 4. The Sequence Object

The `Sequence` object is the central container for every (Py)Pulseq pulse sequence. Usually, we name it `seq` and create it by
passing the system limits from the previous cell. 

In (Py)Pulseq, a sequence is a collection / concatenation of non-overlapping **blocks**, which we will cover later in this tutorial.

In [ ]:
seq = pp.Sequence(system)

---
## 5. Creating Basic Events

PyPulseq provides `make_*` functions to create different types of events. In this
introductory tutorial, we'll start with the simplest ones:

- **`make_block_pulse`** — a rectangular (block) RF pulse
- **`make_adc`** — a readout / ADC event
- **`make_delay`** — a simple delay

These events already allow us to create the most basic sequence, a Free Induction Decay (FID) pulse sequence.

Later tutorials will also cover different types of gradients, shaped RF pulses, and more.

### 5.1 Creating an RF pulse

The first important event for our FID pulse sequence is the RF pulse used to excite the spin system. The simplest RF pulse is a rectangular (block) pulse. 

We specify:
- **`flip_angle`** — the desired flip angle in **radians** (use `np.deg2rad()` to convert from degrees)
- **`duration`** — the pulse duration in seconds
- **`system`** — the system limits object
- **`delay`** — the delay before the pulse in seconds (this has to be >= `system.rf_dead_time`)
- **`use`** — the purpose of the pulse (e.g., `'excitation'` or `'refocusing'`)

In [ ]:
# Create a non-selective 90° block pulse with 1 ms duration
rf_90 = pp.make_block_pulse(
    flip_angle=np.deg2rad(90),
    duration=1e-3,
    system=system,
    delay=system.rf_dead_time,
    use='excitation',
)

### 5.2 Creating a readout / ADC event

The second event we need is an ADC (Analog-to-Digital Converter) event. An ADC event is used to read out the signal from the receiver coil(s).

We specify:

- **`num_samples`** — Number of readout samples
- **`delay`** — Delay before the ADC event in seconds (this has to be >= `system.adc_dead_time`)
- **`dwell`** — The time between two samples in seconds
- **`system`** — System limits

Please note that there are different ways to specify the same ADC event. For example, you can either specify the number of samples and the dwell time, or the number of samples and the total duration. Additionally, parameters such as `freq_offset`, `phase_offset`, `freq_ppm`, and `phase_ppm` can be specified to control the frequency and phase of the ADC event. We don't need these for the simple FID pulse sequence, but we will cover them in later tutorials.

In [ ]:
# Create a simple ADC event with 1024 samples and a dwell time of 20 µs
adc = pp.make_adc(
    num_samples=1024,
    delay=system.adc_dead_time,
    dwell=20e-6,
    system=system,
)

### 5.3 Creating a delay

A delay is simply a period of time where nothing happens. Delays are used to control the timing of the sequence (e.g., to achieve a specific TE or TR). For our simple example, we will create a delay of 100 ms using the `make_delay` function.

In [ ]:
delay = pp.make_delay(100e-3)

---
## 6. Building a Sequence: Blocks and `add_block`

Now let's assemble these events into a sequence. As mentioned above, a Pulseq sequence is a concatenation of non-overlapping **blocks**.

A **block** is a group of different events (e.g. RF pulse, gradient, ADC) that overlap in time within a block. 

Each block may contain:

- one optional gradient per axis
- one optional RF pulse
- one optional ADC event
- one optional delay or soft delay
- one optional trigger
- various optional labels

Individual events within a block may define their own start delays. We already used this feature above when defining the delays of the RF pulse and the ADC event.

The key method to add events to a sequence is `seq.add_block()`. It takes one or more events as arguments and adds them as a block to the sequence.

### 6.1 Building our simple FID sequence

We can now build our simple FID sequence by adding our RF pulse (`rf_90`), the ADC event (`adc`), and the 100 ms delay (`delay`) to the Sequence object (`seq`).

In [ ]:
# Create a new sequence using the system limits
seq = pp.Sequence(system=system)

# Add the RF pulse to the sequence
seq.add_block(rf_90)

# Add the ADC event and the 100 ms delay to the sequence
seq.add_block(adc, delay)

### 6.2 Inspecting event and block durations

The `add_block` method can take a single event or multiple events as arguments. If you pass multiple events, they will be added to the same block, where they overlap in time. Therefore, the duration of a block is given by the longest event in the block. 

Let's investigate the duration of all the events and blocks we created so far using the `calc_duration` function.

In [ ]:
duration_rf_90 = pp.calc_duration(rf_90)
print(f'The duration of the RF pulse is {duration_rf_90 * 1e3:.2f} ms.')

duration_adc = pp.calc_duration(adc)
print(f'The duration of the ADC is {duration_adc * 1e3:.2f} ms.')

duration_delay = pp.calc_duration(delay)
print(f'The duration of the delay is {duration_delay * 1e3:.2f} ms.')

duration_adc_and_delay = pp.calc_duration(adc, delay)
print(f'The duration of the ADC and delay is {duration_adc_and_delay * 1e3:.2f} ms.')

print('')

duration_block1 = seq.block_durations[1]
print(f'The duration of the first block is {duration_block1 * 1e3:.2f} ms.')

duration_block2 = seq.block_durations[2]
print(f'The duration of the second block is {duration_block2 * 1e3:.2f} ms.')

### Durations of the individual events
As you can see, the duration of our 1 ms RF pulse is NOT exactly 1 ms, but 1.12 ms. This is because the total duration of the RF pulse event includes the RF dead time before the RF pulse (`rf_dead_time=100 µs`) and the RF ringdown time after the RF pulse (`rf_ringdown_time=20 µs`).

Something similar can be seen for the total duration of the ADC event. The readout duration is expected to be `num_samples * dwell = 20.48 ms`, but the total duration returned by the `calc_duration` function is 20.50 ms. The additional 20 µs are due to the ADC delay (10 µs from `adc_dead_time`) and rounding to the block duration raster.

For the delay event, the total duration is simply the delay time we specified.

From the last `calc_duration` call, you can see that the function also accepts several events at once. In this case, the returned duration is the maximum duration across all provided events, which is the duration of the delay event (100 ms) in this case.

### Durations of the different blocks
The durations of all blocks of a sequence are accessible via the `seq.block_durations` dictionary, which uses the block numbers as keys and the durations as values.

As we expected, the duration of the first block is given by the total duration (1.12 ms) of the RF pulse event it contains.

The duration of the second block is given by the longest event it contains, which is the delay event (100 ms).

---
## 7. Checking, Plotting, and Exporting

### 7.1 Checking timing

Before using a sequence, it's good practice to check for timing errors. The `check_timing()`
method verifies that all events are properly aligned to the raster times and that there are
no overlapping or conflicting events.

In [ ]:
ok, error_report = seq.check_timing()

if ok:
    print('Timing check passed!')
else:
    print('Timing check failed:')
    print(error_report)

### 7.2 Advanced test report

(Py)Pulseq also provides a more detailed test report that will give you information about the number of different events in your sequence, the total duration, the estimated echo time (TE) and the repetition time (TR), the flip angle(s), the gradient amplitudes and slew rates in the different directions and so on. This is especially useful for more complex sequences, but for completeness, we also show it here.

In [ ]:
print(seq.test_report())

### 7.3 Plotting the sequence

The `seq.plot()` method provides a visual representation of the sequence, showing RF pulses, gradients, and ADC events over time. This can be helpful for debugging and verification, as well as for educational purposes. If we don't specify a time range, the plot will show the entire sequence.

In [ ]:
%matplotlib inline
seq.plot()

However, sometimes it is more helpful to focus on a specific part of the sequence, e.g. the RF excitation pulse and the beginning of the ADC readout in our simple FID sequence. Let's zoom in on this part of the sequence by setting the `time_range` parameter of `seq.plot()` to the first 1.5 ms of the sequence, which allows us to see the RF excitation pulse and its preceding delay (RF dead time) and the first few of the 1024 ADC samples.

In [ ]:
seq.plot(time_range=(0.0, 1.5e-3))

### 7.4 Sequence definitions

Before writing a sequence, you can attach metadata using `seq.set_definition()`. This information is stored in the `.seq` file header and can be used by the scanner or
reconstruction software.

Common definitions include the field of view (FoV), a sequence name, the slice thickness, the echo time (TE), the repetition time (TR), etc. For our FID sequence, we will only define the name to be `simple_fid`, the echo time to be `0.54 ms` (as given in the test report) and a `random_parameter` with a value of `42`. We will go into more detail about which definitions are used by the scanner or reconstruction software in later tutorials.

In [ ]:
seq.set_definition(key='Name', value='simple_fid')
seq.set_definition(key='TE', value=0.54e-3)
seq.set_definition(key='random_parameter', value=42)

### 7.5 Writing a `.seq` file

To export the sequence as a `.seq` file for use on a scanner or simulation software, call `seq.write()`.

In [ ]:
seq.write('simple_fid.seq')

---
## 8. Investigating the `.seq` File

The `.seq` file is a human-readable text file that contains all the information needed to play out the sequence on a scanner. Let's take a look at the content of the `simple_fid.seq` file we just created:

> **`simple_fid.seq`**
> ```
> # Pulseq sequence file
> # Created by PyPulseq
> 
> [VERSION]
> major 1
> minor 5
> revision 0
> 
> [DEFINITIONS]
> AdcRasterTime 1e-07 
> BlockDurationRaster 1e-05 
> GradientRasterTime 1e-05 
> Name simple_fid 
> RadiofrequencyRasterTime 1e-06 
> TE 0.00054 
> TotalDuration 0.10112 
> random_parameter 42 
> 
> # Format of blocks:
> # NUM DUR RF  GX  GY  GZ  ADC  EXT
> [BLOCKS]
> 1 112   1   0   0   0  0  0
> 2 10000   0   0   0   0  1  0
> 
> # Format of RF events:
> # id ampl. mag_id phase_id time_shape_id center delay freqPPm phasePPM freq phase use
> # ..   Hz      ..       ..            ..     us    us     ppm  rad/MHz   Hz   rad  ..
> # Field "use" is the initial of: excitation refocusing inversion saturation preparation other undefined
> [RF]
> 1          250 1 2 3 500 100 0 0 0 0 e
> 
> # Format of ADC events:
> # id num dwell delay freqPPM phasePPM freq phase phase_id
> # ..  ..    ns    us     ppm  rad/MHz   Hz   rad       ..
> [ADC]
> 1 1024 20000 10 0 0 0 0 0
> 
> # Sequence Shapes
> [SHAPES]
> 
> shape_id 1
> num_samples 2
> 1
> 1
> 
> shape_id 2
> num_samples 2
> 0
> 0
> 
> shape_id 3
> num_samples 2
> 0
> 1000
> 
> 
> [SIGNATURE]
> # This is the hash of the Pulseq file, calculated right before the [SIGNATURE] section was added
> # It can be reproduced/verified with md5sum if the file trimmed to the position right above [SIGNATURE]
> # The new line character preceding [SIGNATURE] BELONGS to the signature (and needs to be stripped away for recalculating/verification)
> Type md5
> Hash f955d3c1205157caf089defa15528e7d
> ```

A detailed description of the `.seq` file format can be found in the [specification.pdf](https://pulseq.github.io/specification.pdf) on the Pulseq website.

---
## Summary

In this tutorial, you learned the fundamental building blocks of PyPulseq:

| Concept | Function / Class |
|---|---|
| Hardware limits | `pp.Opts(max_grad=..., max_slew=..., ...)` |
| Sequence container | `pp.Sequence(system)` |
| Block RF pulse | `pp.make_block_pulse(flip_angle=..., duration=..., system=...)` |
| ADC readout | `pp.make_adc(num_samples=..., dwell=..., system=...)` |
| Delay | `pp.make_delay(duration)` |
| Add events to sequence | `seq.add_block(event1, event2, ...)` |
| Compute event duration | `pp.calc_duration(event1, event2, ...)` |
| Check timing | `seq.check_timing()` |
| Test report | `seq.test_report()` |
| Plot | `seq.plot()` |
| Metadata | `seq.set_definition(key=..., value=...)` |
| Export | `seq.write('filename.seq')` |

## Next steps

In the next tutorial, we'll explore **RF pulses** in more detail — including sinc pulses
for slice-selective excitation, and how RF pulses interact with slice-selection gradients.